In [6]:
import os
from task2vec import Task2Vec
from models import get_model
import datasets
import task_similarity
import pickle
import cloudpickle
import torch
import torchvision
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import numpy as np

### pytorchデータセットの設定と作成

In [7]:
DATA_DIRECTORY = 'testdata'
transform = transforms.Compose([
                                transforms.RandomResizedCrop(256, scale=(1,1), ratio=(1,1)),
                                transforms.CenterCrop(256),
                                transforms.ToTensor(), 
                                transforms.Normalize([0.485, 0.456, 0.406],
                                                     [0.229, 0.224, 0.225])
                                ])

train_data = ImageFolder(root=DATA_DIRECTORY, transform=transform)

### タスク埋め込み計算

In [8]:
tasks = []
for i in train_data.class_to_idx.values():
    task = []
    for j, target in enumerate(train_data.targets):
        if i == target:
            task.append(train_data[j])
    tasks.append(task)

In [9]:
embeddings = []
for i, data in enumerate(tasks):
    print(f"Embedding {train_data.classes[i]}")
    probe_network = get_model('resnet34', pretrained=True, num_classes=len(train_data.classes)) #.cuda()
    embedding = Task2Vec(probe_network, max_samples=200, skip_layers=7, label=train_data.classes[i]).embed(data)
    
    #embeddings.append( Task2Vec(probe_network, max_samples=200, skip_layers=7, label=train_data.classes[i]).embed(data) )

Embedding Accipitriformes


TypeError: 'DataLoader' object does not support indexing

In [ ]:
probe_network = get_model('resnet34', pretrained=True, num_classes=len(train_data.classes)) #.cuda()
with open('probe_network.p', 'wb') as ff:
    cloudpickle.dump(probe_network, ff)

In [ ]:
with open('embeddings\\extend-mydata\\0.0004-10-0.0001-6.p', 'wb') as fw:
    pickle.dump(embeddings, fw)

In [ ]:
with open('embeddings\\extend-mydata\\0.0004-10-0.0001-6.p', 'rb') as fr:
    embeddings = pickle.load(fr)

In [ ]:
task_similarity.plot_distance_matrix(embeddings, train_data.classes)

In [ ]:
task_similarity.plot_tsne_embed(embeddings)